In [2]:
#libraries
import os
import geopandas as gpd
import rasterio
import numpy as np


In [16]:
# Path to the folder containing the TIFF files
folder_path = "path_folder_GeoTIFF"

In [17]:
# Get a list of all TIFF files in the folder
tif_files = [f for f in os.listdir(folder_path) if f.endswith(".tif")]

In [18]:
# Function to process a TIFF file and return the hot and cold spots
def process_tiff(tif_path):
    with rasterio.open(tif_path) as src:
        # Read the raster data as a NumPy array
        temperature_matrix = src.read(1)  # Assuming temperature is stored in the first band

        # Calculate quartiles for non-zero values
        non_zero_values = temperature_matrix[temperature_matrix != 0]
        q25 = np.percentile(non_zero_values, 25) 
        q50 = np.percentile(non_zero_values, 50) 
        q75 = np.percentile(non_zero_values, 75) 

        # Find indices of cells of q1, q2, q3, q4
        q4_hot_spot_indices = np.argwhere((temperature_matrix > q75) & (temperature_matrix != 0))
        q3_indices = np.argwhere((temperature_matrix > q50) & (temperature_matrix < q75) & (temperature_matrix != 0))
        q2_indices = np.argwhere((temperature_matrix > q25) & (temperature_matrix < q50) & (temperature_matrix != 0))
        q1_cold_spot_indices = np.argwhere((temperature_matrix < q25) & (temperature_matrix != 0))
        
        return q4_hot_spot_indices, q3_indices, q2_indices, q1_cold_spot_indices

In [19]:
# Dictionary to store hot and cold spot arrays for each TIFF file
q4_hot_spot_results = {}
q3_results = {}
q2_results = {}
q1_cold_spot_results = {}

# Iterate through each TIFF file
for tif_file in tif_files:
    tif_path = os.path.join(folder_path, tif_file)
    
    # Process the TIFF file
    q4_hot_spot_indices, q3_indices, q2_indices, q1_cold_spot_indices = process_tiff(tif_path)

    # Store the hot and cold spot indices in the dictionaries
    q4_hot_spot_results[tif_file] = q4_hot_spot_indices
    q3_results[tif_file] = q3_indices
    q2_results[tif_file] = q2_indices
    q1_cold_spot_results[tif_file] = q1_cold_spot_indices

In [24]:
# Create a dictionary to count the occurrences of each cell position for hot and cold spots
q4_hot_cell_count = {}
q3_cell_count = {}
q2_cell_count =  {}
q1_cold_cell_count = {}

# Iterate through hot and cold spot arrays from TIFF files
#### Q4
for tif_file, hot_spot_indices in q4_hot_spot_results.items():
    for row, col in hot_spot_indices:
        cell_position = (row, col)
        
        # Increment the count for the cell position in hot spots
        if cell_position in q4_hot_cell_count:
            q4_hot_cell_count[cell_position] += 1
        else:
            q4_hot_cell_count[cell_position] = 1

#### Q3
for tif_file, q3_spot_indices in q3_results.items():
    for row, col in q3_spot_indices:
        cell_position = (row, col)
        
        # Increment the count for the cell position in hot spots
        if cell_position in q3_cell_count:
            q3_cell_count[cell_position] += 1
        else:
            q3_cell_count[cell_position] = 1

#### Q2
for tif_file, q2_spot_indices in q2_results.items():
    for row, col in q2_spot_indices:
        cell_position = (row, col)
        
        # Increment the count for the cell position in hot spots
        if cell_position in q2_cell_count:
            q2_cell_count[cell_position] += 1
        else:
            q2_cell_count[cell_position] = 1



#### Q1
for tif_file, cold_spot_indices in q1_cold_spot_results.items():
    for row, col in cold_spot_indices:
        cell_position = (row, col)
        
        # Increment the count for the cell position in cold spots
        if cell_position in q1_cold_cell_count:
            q1_cold_cell_count[cell_position] += 1
        else:
            q1_cold_cell_count[cell_position] = 1

# Create datasets for hot and cold spots that occur more than 10 times
q4_hot_spot_dataset = [(row, col, count) for (row, col), count in q4_hot_cell_count.items() if count > 10]
q3_spot_dataset = [(row, col, count) for (row, col), count in q3_cell_count.items() if count > 10]
q2_spot_dataset = [(row, col, count) for (row, col), count in q2_cell_count.items()if count > 10]
q1_cold_spot_dataset = [(row, col, count) for (row, col), count in q1_cold_cell_count.items() if count > 10]



In [39]:
# Load the original GeoTIFF you want to mark
original_tiff_path = "path_GeoTIFF"
with rasterio.open(original_tiff_path) as src:
    original_data = src.read(1)
    profile = src.profile

In [40]:
# Create arrays for marked spots 
marked_data = np.empty_like(original_data)  

In [41]:
# Define numerical values for the classes
class_values = {
    'Q4': 4,
    'Q3': 3,
    'Q2': 2,
    'Q1': 1,
}

# Mark hot spots in the q4_hot_spot_marked array
for row, col, _ in q4_hot_spot_dataset:
    marked_data[row, col] = class_values['Q4']

# Mark spots in the q3_spot_dataset array
for row, col, _ in q3_spot_dataset:
    marked_data[row, col] = class_values['Q3']

# Mark spots in the q2_spot_dataset array
for row, col, _ in q2_spot_dataset:
    marked_data[row, col] = class_values['Q2']

# Mark cold spots in the q1_cold_spot_marked array
for row, col, _ in q1_cold_spot_dataset:
    marked_data[row, col] = class_values['Q1']


In [43]:
# Save marked data as GeoTIFFs
output_directory = "output_folder" 
output_tiff_path = os.path.join(output_directory, "spots_marked.tif")
with rasterio.open(output_tiff_path, 'w', **src.profile) as dst:
    dst.write(marked_data, 1)